In [8]:
#code num 1
# Import required libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error, mean_absolute_error
from plotly.offline import init_notebook_mode, iplot  # Import for Plotly offline mode
import warnings  # Add this import

warnings.filterwarnings("ignore")

# Initialize Plotly in offline mode
init_notebook_mode(connected=True)

# Load the dataset
df = pd.read_csv(r"C:\Users\4t4\OneDrive\Desktop\time series for beds\holt's-W\WARD_daily.csv")

# Ensure that 'Date' is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Get a list of unique wards
wards = df['WARD'].unique()

# Initialize a DataFrame to store evaluation results for all wards
all_evaluation_results = pd.DataFrame()

# Wards to exclude from the analysis
excluded_wards = ["KHCC-ADULT DAY CASE NZR", "KHCC-EMERGENCY KSB ROOM", "KHCC-BMT DAY CASE"]

# Iterate through each ward
for ward in wards:
    # Skip the excluded wards
    if ward in excluded_wards:
        print(f"Skipping ward: {ward}")
        continue

    print(f"\nProcessing ward: {ward}")

    # Filter the DataFrame to include only the data for the current ward
    df_ward = df[df['WARD'] == ward][['Date', 'TotalBedsUsed']].copy()

    # Check if there is enough data (at least 187 days)
    if len(df_ward) < 187:  # Need at least 7 points for forecasting and 180 points for testing
        print(f"Not enough data for ward {ward}. Skipping.")
        continue

    # Sort the DataFrame by 'Date'
    df_ward.sort_values('Date', inplace=True)

    # Set 'Date' as the index
    df_ward.set_index('Date', inplace=True)

    # Extract the 'TotalBedsUsed' series
    total_beds = df_ward['TotalBedsUsed']

    # Split the data: use all but last 180 days for training, last 180 days for testing
    train_data = total_beds.iloc[:-180]
    test_data = total_beds.iloc[-180:]

    # Initialize an empty DataFrame to store the forecasts and actuals for 180 days
    forecast_results = pd.DataFrame(index=test_data.index, columns=['Actual', 'Additive', 'Additive_Damped', 'Multiplicative', 'Multiplicative_Damped'])

    # Set actual values for comparison
    forecast_results['Actual'] = test_data

    # Step 2: Fit the initial Holt-Winters models on the training data
    try:
        # Fit four different Holt-Winters models:
        # 1. Additive trend and seasonality
        fit1 = ExponentialSmoothing(
            train_data,
            seasonal_periods=7,
            trend="add",
            seasonal="add",
            use_boxcox=True,
            initialization_method="estimated",
        ).fit()

        # 2. Additive trend and seasonality with damping
        fit3 = ExponentialSmoothing(
            train_data,
            seasonal_periods=7,
            trend="add",
            seasonal="add",
            damped_trend=True,
            use_boxcox=True,
            initialization_method="estimated",
        ).fit()

        # 3. Additive trend and multiplicative seasonality
        fit2 = ExponentialSmoothing(
            train_data,
            seasonal_periods=7,
            trend="add",
            seasonal="mul",
            use_boxcox=True,
            initialization_method="estimated",
        ).fit()

        # 4. Additive trend and multiplicative seasonality with damping
        fit4 = ExponentialSmoothing(
            train_data,
            seasonal_periods=7,
            trend="add",
            seasonal="mul",
            damped_trend=True,
            use_boxcox=True,
            initialization_method="estimated",
        ).fit()

    except Exception as e:
        print(f"Error fitting models for ward {ward}: {e}")
        continue

    # Step 3: Perform forecasting over the 180 days using actual values
    for i in range(0, 180, 7):
        # Update training data to include actual values up to the current point
        temp_train_data = total_beds.iloc[:-(180 - i)]
        
        # Step 4: Refit the models on the updated train data
        try:
            # Refit all four models with updated data
            fit1 = ExponentialSmoothing(
                temp_train_data,
                seasonal_periods=7,
                trend="add",
                seasonal="add",
                use_boxcox=True,
                initialization_method="estimated",
            ).fit()

            fit3 = ExponentialSmoothing(
                temp_train_data,
                seasonal_periods=7,
                trend="add",
                seasonal="add",
                damped_trend=True,
                use_boxcox=True,
                initialization_method="estimated",
            ).fit()

            fit2 = ExponentialSmoothing(
                temp_train_data,
                seasonal_periods=7,
                trend="add",
                seasonal="mul",
                use_boxcox=True,
                initialization_method="estimated",
            ).fit()

            fit4 = ExponentialSmoothing(
                temp_train_data,
                seasonal_periods=7,
                trend="add",
                seasonal="mul",
                damped_trend=True,
                use_boxcox=True,
                initialization_method="estimated",
            ).fit()

        except Exception as e:
            print(f"Error refitting models for ward {ward}: {e}")
            continue

        # Forecast the next 7 days using actual values
        try:
            # Generate forecasts for all four models
            hw_forecast_additive = fit1.forecast(7)
            hw_forecast_multiplicative = fit2.forecast(7)
            hw_forecast_additive_dam = fit3.forecast(7)
            hw_forecast_multiplicative_dam = fit4.forecast(7)

            # Determine the number of days left to forecast
            days_left = min(7, len(forecast_results) - i)

            # Store forecasts in the results DataFrame, using only the available days
            forecast_results['Additive'].iloc[i:i+days_left] = hw_forecast_additive.values[:days_left]
            forecast_results['Multiplicative'].iloc[i:i+days_left] = hw_forecast_multiplicative.values[:days_left]
            forecast_results['Additive_Damped'].iloc[i:i+days_left] = hw_forecast_additive_dam.values[:days_left]
            forecast_results['Multiplicative_Damped'].iloc[i:i+days_left] = hw_forecast_multiplicative_dam.values[:days_left]

        except Exception as e:
            print(f"Error forecasting for ward {ward}: {e}")
            continue

    # After the loop, remove any rows with NaN values
    forecast_results = forecast_results.dropna()

    # Step 5: Plot the 180-day forecast vs actual values using Plotly offline mode
    fig = go.Figure()

    # Add the actual data to the plot
    fig.add_trace(go.Scatter(
        x=forecast_results.index, y=forecast_results['Actual'], mode='lines+markers',
        name='Actual Data', line=dict(color='black')
    ))

    # Add the forecasted data for all four models to the plot
    fig.add_trace(go.Scatter(
        x=forecast_results.index, y=forecast_results['Additive'], mode='lines+markers',
        name='Additive Forecast', line=dict(color='red', dash='dash')
    ))

    fig.add_trace(go.Scatter(
        x=forecast_results.index, y=forecast_results['Multiplicative'], mode='lines+markers',
        name='Multiplicative Forecast', line=dict(color='green', dash='dash')
    ))

    fig.add_trace(go.Scatter(
        x=forecast_results.index, y=forecast_results['Additive_Damped'], mode='lines+markers',
        name='Additive Damped Forecast', line=dict(color='blue', dash='dash')
    ))

    fig.add_trace(go.Scatter(
        x=forecast_results.index, y=forecast_results['Multiplicative_Damped'], mode='lines+markers',
        name='Multiplicative Damped Forecast', line=dict(color='orange', dash='dash')
    ))

    # Update layout of the plot
    fig.update_layout(
        title=f"Actual vs Forecast (180 Days) for Ward: {ward}",
        xaxis_title='Date',
        yaxis_title='Total Beds Used',
        showlegend=True
    )

    # Display the plot interactively using Plotly's offline mode
    iplot(fig)

    # Step 6: Calculate MSE and MAE for the 180-day forecast
    try:
        # Calculate MSE and MAE for each model
        mse_results = pd.DataFrame({
            'Model': ['Additive', 'Additive Damped', 'Multiplicative', 'Multiplicative Damped'],
            'MSE': [
                mean_squared_error(forecast_results['Actual'], forecast_results['Additive']),
                mean_squared_error(forecast_results['Actual'], forecast_results['Additive_Damped']),
                mean_squared_error(forecast_results['Actual'], forecast_results['Multiplicative']),
                mean_squared_error(forecast_results['Actual'], forecast_results['Multiplicative_Damped'])
            ],
            'MAE': [
                mean_absolute_error(forecast_results['Actual'], forecast_results['Additive']),
                mean_absolute_error(forecast_results['Actual'], forecast_results['Additive_Damped']),
                mean_absolute_error(forecast_results['Actual'], forecast_results['Multiplicative']),
                mean_absolute_error(forecast_results['Actual'], forecast_results['Multiplicative_Damped'])
            ]
        })

        # Add the current ward to the results and append to the overall results DataFrame
        mse_results['Ward'] = ward
        all_evaluation_results = pd.concat([all_evaluation_results, mse_results], ignore_index=True)

    except Exception as e:
        print(f"Error calculating MSE and MAE for ward {ward}: {e}")

# Save the evaluation results for all wards to a CSV file
all_evaluation_results.to_csv('evaluation_results_all_wards.csv', index=False)
print("\nEvaluation results saved to 'evaluation_results_all_wards.csv'")


Processing ward: KHCC-10H-BMT



Processing ward: KHCC-10H-MEDICAL/SURGICAL



Processing ward: KHCC-11H-MEDICAL/SURGICAL



Processing ward: KHCC-2A-MEDICAL/SURGICAL



Processing ward: KHCC-2B-AICU


...


Processing ward: KHCC-ADULT DAY CASE KSB


Skipping ward: KHCC-EMERGENCY KSB ROOM

Processing ward: KHCC-EMERGENCY ROOM



Processing ward: KHCC-PED DAY CASE


Skipping ward: KHCC-BMT DAY CASE
Skipping ward: KHCC-ADULT DAY CASE NZR

Processing ward: KHCC-5C-KOLAGHASSI WING



Evaluation results saved to 'evaluation_results_all_wards.csv'


### Recursive Process: For each 7-day forecasting window, the models are refit using the real observed data up to that point.

In [38]:
import pandas as pd
import plotly.graph_objects as go

# Load the evaluation results from the CSV file
all_evaluation_results = pd.read_csv(r"C:\Users\4t4\OneDrive\Desktop\time series for beds\holt's-W\evaluation_results_all_wards_holts.csv")

# Get the list of unique wards
wards = all_evaluation_results['Ward'].unique()

# Loop through each ward and create a plot for both MSE and MAE
for ward in wards:
    ward_data = all_evaluation_results[all_evaluation_results['Ward'] == ward]

    # Create a figure for this ward
    fig = go.Figure()

    # Add MSE bars for each model
    fig.add_trace(go.Bar(
        x=ward_data['Model'],
        y=ward_data['MSE'],
        name='MSE',
        text=ward_data['MSE'],
        textposition='auto',
        marker=dict(color='blue')
    ))

    # Add MAE bars for each model
    fig.add_trace(go.Bar(
        x=ward_data['Model'],
        y=ward_data['MAE'],
        name='MAE',
        text=ward_data['MAE'],
        textposition='auto',
        marker=dict(color='orange')
    ))

    # Update layout to group bars by model for this ward
    fig.update_layout(
        title=f'Error Metrics for Ward: {ward}',
        xaxis_title='Model',
        yaxis_title='Error Value',
        barmode='group',  # Group MSE and MAE bars side by side
        showlegend=True
    )

    # Show the plot for the current ward
    fig.show()
